<h1>Numpy and Pandas Home Assignment</h1>
In this assignment, we will use pandas and numpy to do some fairly basic analysis on equities.

<h3>Imports</h3>

In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import pandas_datareader.data as web
#import fix_yahoo_finance as yf
import yfinance as yf

/Users/Zhang/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


<h3>Get data on the following four stocks from yahoo finance</h3>
<li>MS, GS, JPM, C
<li>Use the date range 2013/1/1/ to 2018/9/24

In [2]:
start = datetime.datetime(2013,1,1)
end = datetime.datetime(2018,9,24)
stocks = web.get_data_yahoo(["MS","GS","JPM","C"],start,end)
stocks = stocks['Close']
# df = web.DataReader(['MS', 'GS','JPM','C'],'yahoo', start,end)['Close']
stocks    # a pandas.DataFrame
# df

Symbols,MS,GS,JPM,C
Date,,,,
2013-01-02,19.620001,131.660004,44.660000,41.250000
2013-01-03,19.580000,130.940002,44.570000,41.389999
2013-01-04,20.190001,134.509995,45.360001,42.430000
2013-01-07,19.799999,134.259995,45.410000,42.470001
2013-01-08,19.650000,133.050003,45.500000,42.459999
...,...,...,...,...
2018-09-18,47.849998,228.889999,114.300003,71.360001
2018-09-19,49.099998,235.580002,117.620003,73.720001
2018-09-20,49.880001,237.399994,118.629997,74.790001


<h3>Compute the cross correlation coefficients on the stocks</h3>
<li>Create a df 'rets' with the one day percentage changes
<li>Calculate cross correlations on the rets df

In [3]:
rets = stocks.pct_change()
rets_correlation = rets.corr()
rets_correlation

Symbols,MS,GS,JPM,C
Symbols,,,,
MS,1.000000,0.844340,0.801232,0.809949
GS,0.844340,1.000000,0.813289,0.792073
JPM,0.801232,0.813289,1.000000,0.849607
C,0.809949,0.792073,0.849607,1.000000


<h3>Select a pair of stocks to trade</h3>
<li>Choose the pair with the highest correlation
<li>You can just eyeball the result and pick the highest correlation pairs
<li>Create a new df 'pairs' with the returns columns of the highest correlation pairs

In [4]:
# The pair with the highest correlation is 'JPM' & 'C' with 0.849616.
pairs = rets[['JPM','C']]
pairs   # a pandas.series

Symbols,JPM,C
Date,,
2013-01-02,NaN,NaN
2013-01-03,-0.002015,0.003394
2013-01-04,0.017725,0.025127
2013-01-07,0.001102,0.000943
2013-01-08,0.001982,-0.000236
...,...,...
2018-09-18,0.004041,0.006772
2018-09-19,0.029046,0.033072
2018-09-20,0.008587,0.014514


<h3>Calculate the mean and std dev of the columns (returns of the two stocks)</h3>
<li>Note that line 1 contains a NaN. You'll have to get rid of this line
<li>df.describe() returns summary stats for all cols in a dataframe. You can use this to extract means and standard deviations

In [5]:
new_df = rets[['C','JPM']].iloc[1:-1]  # to get rid of the 1st line of 'rets'
c_std = new_df.describe()['C']['std']
j_std = new_df.describe()['JPM']['std']
c_mean = new_df.describe()['C']['mean']
j_mean = new_df.describe()['JPM']['mean']
new_df.describe()

Symbols,C,JPM
count,1441.000000,1441.000000
mean,0.000518,0.000756
std,0.014858,0.012830
min,-0.093567,-0.069477
25%,-0.007248,-0.005654
50%,0.000386,0.000493
75%,0.008252,0.007661
max,0.073185,0.083286


<h3>Create a new column "Long_JPM_Short_C"</h3>
<li>If the return of C is greater than its mean + 0.25* std
<li> AND
<li>If the return of JPM is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [6]:
u = c_mean+(c_std*0.25)
v = j_mean-(j_std*0.25)
new_df['Long_JPM_Short_C'] = np.where((new_df['C']>u) & (new_df['JPM']<v),True,False)
new_df

Symbols,C,JPM,Long_JPM_Short_C
Date,,,
2013-01-03,0.003394,-0.002015,False
2013-01-04,0.025127,0.017725,False
2013-01-07,0.000943,0.001102,False
2013-01-08,-0.000236,0.001982,False
2013-01-09,-0.009892,-0.000659,False
...,...,...,...
2018-09-17,0.004820,0.002996,False
2018-09-18,0.006772,0.004041,False
2018-09-19,0.033072,0.029046,False


<h3>Create a new column "Long_C_Short_JPM"</h3>
<li>If the return of JPM is greater than its mean + 0.25* std
<li> AND
<li>If the return of C is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [7]:
x = j_mean+(j_std*0.25)
y = c_mean-(c_std*0.25)
new_df['Long_C_Short_JPM'] = np.where((new_df['JPM']>x) & (new_df['C']<y),True,False)
new_df

Symbols,C,JPM,Long_JPM_Short_C,Long_C_Short_JPM
Date,,,,
2013-01-03,0.003394,-0.002015,False,False
2013-01-04,0.025127,0.017725,False,False
2013-01-07,0.000943,0.001102,False,False
2013-01-08,-0.000236,0.001982,False,False
2013-01-09,-0.009892,-0.000659,False,False
...,...,...,...,...
2018-09-17,0.004820,0.002996,False,False
2018-09-18,0.006772,0.004041,False,False
2018-09-19,0.033072,0.029046,False,False


<h3>Create a new column "Trade_return"</h3>
<li>If Long_JPM_Short_C is True and Long_C_Short_JPM is False, the value of this column should be the return on JPM minus the return on C
<li>If Long_JPM_Short_C is False and Long_C_Short_JPM is True, the value of this column should be the return on C minus the return on JPM
<li>Otherwise the value should be zero

In [8]:
new_df['Trade_return'] = 0
C_rets = new_df['C']
JPM_rets = new_df['JPM']
new_df.loc[((new_df['Long_JPM_Short_C'] == True) & (new_df['Long_C_Short_JPM'] == False)),'Trade_return'] = JPM_rets - C_rets
new_df.loc[((new_df['Long_JPM_Short_C'] == False) & (new_df['Long_C_Short_JPM'] == True)),'Trade_return'] = C_rets - JPM_rets

In [9]:
new_df

Symbols,C,JPM,Long_JPM_Short_C,Long_C_Short_JPM,Trade_return
Date,,,,,
2013-01-03,0.003394,-0.002015,False,False,0.0
2013-01-04,0.025127,0.017725,False,False,0.0
2013-01-07,0.000943,0.001102,False,False,0.0
2013-01-08,-0.000236,0.001982,False,False,0.0
2013-01-09,-0.009892,-0.000659,False,False,0.0
...,...,...,...,...,...
2018-09-17,0.004820,0.002996,False,False,0.0
2018-09-18,0.006772,0.004041,False,False,0.0
2018-09-19,0.033072,0.029046,False,False,0.0


<h3>Calculate the sum of this new column</h3>
<li>Are you going to get rich?

In [10]:
earnings = sum(new_df['Trade_return'])
earnings

-0.6793516982785628

In [11]:
# The sum of Trade Returns is negative, so no, I am not going to get rich by investing in 
# the highly correlated pair of 'JPM' and 'C'.